In [ ]:
import json
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import binom
from scipy.stats import gamma
from scipy.special import gamma as gamma_fn
import re
import os
from matplotlib import pyplot as plt
from typing import Tuple, List

from distutils.dir_util import copy_tree

In [ ]:
from run_config_json import create_run_config_json

In [ ]:
STATE_NAMES_KEY = "stateTimeMap"
TOTAL_TIME_KEY = "t_total"

In [ ]:
# General settings
num_runs = 1
starting_seed = 0
seed_multiplier = 100

# Validator settings
num_nodes = 16
num_consensus = 5000
base_time_limit = 10000
node_processing_rate = 3
consensus_protocol = "HS"

## Fault settings
num_faults = 1
fault_type = "UR"
fault_parameters = []

# Network settings
## Switch settings
switch_processing_rate = -1
message_channel_success_rate = 1

network_type = "Clique"
network_parameters = []

In [ ]:
VALIDATOR_RESULTS_FILEPATH = "../json"
RESULTS_DIRECTORY = "json_n{num_nodes}_btl{base_time_limit:.1f}_{topology}_{topo_params}_{switch_processing_rate:.1f}_{protocol}_{num_faults}_{fault_type}_{fault_params}"

In [ ]:
def process_results(results_dic: dict): 
    processed_results_dic = {}
    state_names = None
    for var in results_dic.keys():
        temp_dic = results_dic[var].copy()
        if state_names == None:
            state_names = temp_dic[STATE_NAMES_KEY].keys()
        for key in temp_dic[STATE_NAMES_KEY].keys():
            temp_dic[key] = temp_dic[STATE_NAMES_KEY][key]
        del temp_dic[STATE_NAMES_KEY]
        processed_results_dic[var] = temp_dic
    return (processed_results_dic, state_names) 

def write_str_to_file(file_string: str, filename: str) -> None:
    with open(filename, "w") as file:
        file.write(file_string)

def run_and_save(run_config_dic: str, output_directory: str) -> None:
    config_filename = "config.json"
    write_str_to_file(run_config_dic, config_filename)

    argument = "py/" + config_filename
    ! (cd "../" && gradlew run --args={argument})
    copy_tree(VALIDATOR_RESULTS_FILEPATH, output_directory)    

def construct_results_directory(num_nodes: int, base_time_limit: float, topology: str, topo_params: List[int], switch_processing_rate: float,
                                protocol: str, num_faults: int, fault_type: str, fault_params: List[int]) -> str:
    return RESULTS_DIRECTORY.format(num_nodes=num_nodes,  base_time_limit=base_time_limit, topology=topology, topo_params=topo_params, switch_processing_rate=switch_processing_rate, 
                                    protocol=protocol, num_faults=num_faults, fault_type=fault_type, fault_params=fault_params)


In [ ]:
results_dic = {}
# for base_time_limit in [10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18, 18.5, 19, 19.5, 20]:
# for base_time_limit in [20.5, 21, 21.5, 22, 22.5, 23, 24, 25, 26, 27, 28, 29, 30]: 
for base_time_limit in range(31, 51):
# for base_time_limit in [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5]:
# for num_nodes in [4, 8, 16, 24, 32, 48, 64]:
    json_obj = create_run_config_json(num_runs, starting_seed, seed_multiplier,
                                      num_nodes, num_consensus, base_time_limit, node_processing_rate, consensus_protocol,
                                      num_faults, fault_type, fault_parameters,
                                      switch_processing_rate, message_channel_success_rate, network_type, network_parameters)
    run_and_save(json_obj, construct_results_directory(num_nodes, float(base_time_limit), network_type.lower(), network_parameters, switch_processing_rate, consensus_protocol.lower(),
                                                       num_faults, fault_type, fault_parameters))


In [ ]:
RESULTS_FOLDER_REGEX = r'json_n(.+)_btl(.+)_(.+)_(.+)_(.+)_(.+)_(.+)_(.+)_(.+)'
RESULTS_VALIDATOR_FILENAME = "validator_results.json"
RESULTS_DIRECTORY = "results"
FASTEST_MESSAGE_MAP = "fastestMessageCountMap"
REMAINDER_MESSAGE_MAP = "remainderMessageCountMap"
FASTEST_TIME_MAP = "fastestStateTimeMap"
REMAINDER_TIME_MAP = "remainderStateTimeMap"
PREPARED = "PREPARED"
PREPREPARED = "PREPREPARED"
COMMIT = "COMMIT"
SYNC = "SYNC"
ROUND_CHANGE = "ROUND_CHANGE"
TOTAL_TIME_KEY = "t_total_fastest"
RC_PROB = "RC_PROB"
NEW_ROUND = "NEW_ROUND"
PRE_PREPARED = "PRE_PREPARED"
LAMBDA_FASTEST = "lambda_fastest"
L_FASTEST = "L_fastest"
L_REMAINDER = "L_remainder"

NEW_VIEW = "NEW_VIEW"
PREPARE = "PREPARE"
PRE_COMMIT = "PRE_COMMIT"
DECIDE = "DECIDE"
COMMIT = "COMMIT"

IBFT_STATES = [NEW_ROUND, PRE_PREPARED, PREPARED, ROUND_CHANGE]
HS_STATES = [PREPARE, PRE_COMMIT, COMMIT, DECIDE]
PROTOCOL_NAME_STATE_MAP = {"hs": HS_STATES, "ibft": IBFT_STATES}

In [ ]:
results_lst = os.listdir("results/")
jsons = []
index = []
t_total = []
states = PROTOCOL_NAME_STATE_MAP[consensus_protocol.lower()] 
fastest_state_times = {state : [] for state in states}
remainder_state_times = {state : [] for state in states}
fastest_message_arrival_rates = []

fastest_message_queue_lengths = []
remainder_message_queue_lengths = []
for result_filename in results_lst:
    matcher = re.match(RESULTS_FOLDER_REGEX, result_filename)
    if matcher == None: 
        continue
    run_num_nodes = int(matcher.group(1))
    run_base_time_limit = float(matcher.group(2))
    run_topology = matcher.group(3)
    run_topo_params = matcher.group(4) # str instead of a list, not usable 
    run_switch_processing_rate = float(matcher.group(5))
    run_protocol = matcher.group(6)
    run_num_faults = int(matcher.group(7))
    run_fault_type = matcher.group(8)
    run_fault_params = matcher.group(9) # str instead of a list, not usable 

    if run_protocol != consensus_protocol.lower() or run_num_nodes != num_nodes or run_num_faults != num_faults:
        continue

    index.append(run_base_time_limit)
    with open(os.path.join(RESULTS_DIRECTORY, result_filename, RESULTS_VALIDATOR_FILENAME), "r") as json_result:
        result_json = json.load(json_result)
        jsons.append(result_json)
        t_total.append(result_json[TOTAL_TIME_KEY])
        fastest_message_arrival_rates.append(result_json[LAMBDA_FASTEST])
        fastest_message_queue_lengths.append(result_json[L_FASTEST])
        remainder_message_queue_lengths.append(result_json[L_REMAINDER])
        for state in states:
            fastest_state_times[state].append(result_json[FASTEST_TIME_MAP][state])
            remainder_state_times[state].append(result_json[REMAINDER_TIME_MAP][state])

df = pd.DataFrame({"t_total": t_total}, index=index)
df.sort_index(inplace=True)
df["t_total"].plot(grid=True, style=".-", xlabel="base_time_limit", y="time_to_consensus", title=consensus_protocol + " simulation")
plt.show()
df = pd.DataFrame(fastest_state_times, index=index)
df["t_total"] = t_total
df.sort_index(inplace=True)
df.plot(grid=True, style=".-")
df

In [ ]:
def hs_time(n, t, mu, te2e):
    f = (n - 1) // 3
    p = gamma.cdf(t, 4 * n + 3 - f, scale=(1/mu))
    # print(p)
    t_fail = (1 - p) * t 
    # p2 = gamma.cdf(2 * t, 4 * n + 3  - f, scale=(1/mu))
    # t_fail += (1 - p) * (1 - p2) * (2 * t)
    t_succeed = 1 * te2e
    # p3 = gamma.cdf(4 * t, 4 * n + 3 - f, scale=(1/mu))
    # t_fail += (1 - p) * (1 - p2) * (1 - p3) * (4 * t)
    t_total = t_fail + t_succeed
    return t_total

def hs_time_2(n, t, mu, te2e):
    f = (n - 1) // 3
    alpha = 4 * n + 3 - f
    beta = mu
    mode = (alpha - 1) / beta
    gradient = beta ** alpha / gamma_fn(alpha) * mode ** (alpha - 1) * np.exp(-beta * mode)
    intercept = gamma.cdf(mode, alpha, scale=1/mu) - gradient * mode

    def linear_approx(t):
        return 1 - min(1, max(0, intercept + gradient * t))

    return linear_approx(t) * t + te2e

def hs_time_3(n, t, mu, te2e):
    f = (n - 1) // 3
    alpha = 4 * n + 3 - f
    total = 0
    while t * mu < alpha:
        total += t
        t *= 2
    total += alpha / mu
    return total


df["prediction"] = df.index.map(lambda t: hs_time_3(16, t, 10/3, 21))
df.plot(y=["prediction", "t_total"], style=".-", figsize=(10, 5), grid=True, xlabel="base_time_limit", ylabel="time per consensus")




In [ ]:
### IBFT WORK DO NOT TOUCH
round_change_probs = np.array(map(lambda dic: min(dic[FASTEST_MESSAGE_MAP][PREPREPARED] - 1, 1), jsons))
consensus_times = np.array(map(lambda dic: dic[TOTAL_TIME_KEY], jsons))

dic = {RC_PROB: round_change_probs, TOTAL_TIME_KEY: consensus_times, ROUND_CHANGE: fastest_state_times[ROUND_CHANGE], "COMBINED": fastest_state_times["COMBINED"],"z_value": ((np.array(index) - 10) / 2.5)}

df = pd.DataFrame(dic, index=index)
df.sort_index(inplace=True)
df["RC_PROB"].plot(style=".-", grid=True, xlabel="base time limit", ylabel="probability of round change")
df["z_value"].apply(lambda x: 1 - norm.cdf(x)).plot()
###
def time3(t):
    def helper(t, message_penalty):
        if t >= 20:
            return 12 
        elif t <= 4:
            return t + helper(2 * t, max(0, message_penalty - int(t * 3)) + 16)
        p = norm.cdf((t - message_penalty / 3 - 10) / 2.5)
        return (1 - p) * (8 + t + helper(2 * t, 5)) + p * min(12, t)
    return helper(t, 0) - 2.7 + calc(t, 3)

###
# pd.Series(range(1, 40)).map(time3).plot()
df["prediction"] = df.index.map(time3)
df[[TOTAL_TIME_KEY, "prediction"]].plot(grid=True, style=".-")

### break
rho = 2.67 / 3
def pi(i):
    return (1 - rho) * pow(rho, i)

def calc(t, processing_rate):
    i = 0
    E = 0
    cdf = 0
    while i / processing_rate <= t:
        E += (i / processing_rate) * pi(i)
        cdf += pi(i)
        i += 1
    E += (1 - cdf) * t 
    p = (1 - norm.cdf((t - 12)))
    p2 = 4 * p / 16
    E = E * (1 - p2) + p2 * t
    return E

pd.DataFrame({ROUND_CHANGE: fastest_state_times[NEW_ROUND], "TEST": map(lambda x : calc(x, 3), index)}, index=index).sort_index().plot(grid=True, style=".-")